#### Trabajo Práctico N° 2
##### Sistemas de Recomendación
Facultad de Ciencias Exactas y Naturales

Álgebra Lineal Computacional 2°C 2023

Luca Jaichenco ~ 591/22 |Mario Sigal Aguirre ~ 157/22

In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Ejercicio 1: Pasos para implementar un sistema de recomendación de vinos

Se siguieron los pasos descritos en el trabajo practico y en el archivo auxiliar para la creación de un sistema de recomendación de vinos.

#### (a) Descargar los datos

In [3]:
df_wine = pd.read_csv('./wine.csv')

#### (b) Separar los datos en variables independientes (X) y dependiente (Y).

In [4]:
# Variables independientes:
variables_independientes = ['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium',
                            'Total_Phenols', 'Flavanoids', 'Nonflavanoid_Phenols',
                            'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']
df_wine_independientes = df_wine[variables_independientes]

# Variable dependiente:
variables_dependientes =['Customer_Segment']
df_wine_dependientes = df_wine[variables_dependientes]

#### (c) Normalizar y centrar los datos.
Se pide construir una función en python que normalice y centre los valores respecto del promedio. Es decir, aplicar a cada variable independiente:
1. $ xi = (X_i - \bar{X})/s $ 

2. $ \bar{X} = (\sum_{i=1}^{N} X_i) /N  $

3. $ s = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} (X_i - \bar{X})^2} $

In [5]:
#función que normaliza y centra respecto del promedio 
def normalizacion(X):# recibe un Data Frame que contenga solamente las variables independientes    
    
    # inicialización de variables
    mean = np.mean(X, axis=0)
    s = np.std(X, axis=0)
    X_norm = (X - mean) / s
    
    return X_norm, mean, s

In [14]:
df_wine_independientes_norm, mean, s = normalizacion(df_wine_independientes)
X_norm.mean(axis=0)

Alcohol                -8.382808e-16
Malic_Acid             -1.197544e-16
Ash                    -8.370333e-16
Ash_Alcanity           -3.991813e-17
Magnesium              -3.991813e-17
Total_Phenols           0.000000e+00
Flavanoids             -3.991813e-16
Nonflavanoid_Phenols    3.592632e-16
Proanthocyanins        -1.197544e-16
Color_Intensity         2.494883e-17
Hue                     1.995907e-16
OD280                   3.193450e-16
Proline                -1.596725e-16
dtype: float64

Normalizar un conjunto de datos es importante por varias razones:
En principio queremos que las variables tengan las mismas escala y mismo rango, pues de no tenerlas, la matriz de covarianza podría verse afectada, dando más peso a las variables con escalas más grandes y haciendo que las variables con rangos más pequeños tengan una contribución mínima. La normalización asegura que todas las variables tengan la misma escala y rango.
A su ves  la normalización facilita la interpretación de la matriz de covarianza y de los resultados de PCA ya que como los componentes principales proporcionan información sobre la contribución de cada variable a la variabilidad total, normalizar hace que nos sea posible comparar estas contribuciones de manera optima.

#### (d) Calcular la matriz de covarianza
$cov(b) = \frac{b^{*}b}{n-1}$ 

In [15]:
#pasamos el data frame a una matriz
matriz_wine_norm = df_wine_independientes_norm.values

#calculamos la matriz de covarianza
wine_cov = (matriz_wine_norm.T @ matriz_wine_norm)/177

La matriz de covarianza describe la relación entre las variables de nuestro data frame. Los elementos en la posición $(i,j)$  de la matriz se encuentran entre menos uno y uno [-1,1] y representan la covarianza entre las variables $ X_i$ y $X_j$: 
Si el elemento es cercano a uno indica una relación positiva, Cuando $X_i$ tiende a ser grande, $X_j$ también tiende a ser grande, y viceversa.
Si el elemento es cercano a menos uno indica una relación negativa. Cuando $X_i$ tiende a ser grande,$X_j$ tiende a ser pequeño, y viceversa.
Si el elemento es cercano a cero indica que no existe una relación precisa 
Es de dimension $n \times n$ y en nuestro caso $ 13 \times 13. cumple las propiedades de ser Simétrica y Definida Positiva

#### (e) Encontrar el máximo autovalor y el correspondiente autovector de la matriz de covarianza  por el método de la potencia

In [8]:

def metodo_potencia(m, its=100):
  
    v = np.random.rand(m.shape[0])  # Inicializar un vector aleatorio
    for i in range(its):
        v = np.dot(m, v)
        v = v / np.linalg.norm(v)  # Normalizar el vector en cada iteración
    
    AutoValor = np.dot(np.dot(m, v), v) / np.dot(v, v) 
    AutoVector = v
    return AutoValor, AutoVector

# max_Aval_wine_cov, Avec_de_max_Aval_wine_cov = metodo_potencia(wine_cov)

# print(max_Aval_wine_cov, Avec_de_max_Aval_wine_cov)

In [20]:
#(f)
def todos_Avals_Avecs(A, n):
    Avals = []
    Avects = []
    Aval = 0
    Avect = np.zeros(A.shape[0])
    for i in range(0,n):
        A = A - Aval*np.outer(Avect,Avect)
        Aval, Avect = metodo_potencia(A)
        Avals.append(round(Aval, 5))
        Avects.append(Avect)
    return Avals, Avects


In [ ]:
#(g)
# buscamos autovalores y autovectores de la matriz de covarianza
# Avals, Avects = todos_Avals_Avects(wine_cov, 13)

# Sea V  la matriz con los autovectores por columnas  
# V = np.array(Avects).T
# # Sea D la matriz diagonal con los autovalores en la diagonal ordenados de mayor a menor
# D = np.diag(Avals)

# decidimos tomar como los autovalores mas relevantes a los primeros 3

In [21]:
def distancia(v1,v2):
    x = 0
    for i in  range(v1.shape[0]):
        x += (v1[i]-v2[i])**2
    return np.sqrt(x)


def KNN(T_clasif, T_train, Y_train, K): # devuelve Y_estimado
    Y_estimado = []
    for i in range(len(T_clasif)):
        # Calculo las distancias
        distancias = []
        for j in range(len(T_train)):
            distancias.append((distancia(T_clasif[i],T_train[j]), j))
        # Ordeno las distancias, manteniendo el indice de la fila
        distancias.sort()
        # Calculo los resultados
        resultados = {
            1:0,
            2:0,
            3:0
        }
        for k in range(K):
            resultados[Y_train[distancias[k][1]]] += 1
        # Agrego el resultado que mas veces apareció en los k vecinos mas cercanos
        Y_estimado.append(max(resultados, key=resultados.get))
    return Y_estimado

In [12]:
# Separo los datos en train y test
from sklearn.model_selection import train_test_split

In [23]:
# Dividimos el dataframe normalizado en casos de train y en casos de test
X_train, X_test, Y_train, Y_test = train_test_split(df_wine_independientes_norm, df_wine_dependientes, test_size=0.2, random_state=0)

# Definimos la matriz de covarianza para los casos de train
X_train_cov = (X_train.T @ X_train)/len(X_train)

# buscamos autovalores y autovectores de la matriz de covarianza
Avals, Avects = todos_Avals_Avecs(X_train_cov, 13)

# Sea V  la matriz con los autovectores por columnas  
V = np.array(Avects).T

CP = 4 # Cantidad de componentes principales
K = 5 # Nearest neigbours

# Calculo las componentes principales
T_train = (X_train @ V[:,:CP]).values # el .values convierte el df en un np array
T_test = (X_test @ V[:,:CP]).values

Y_train = Y_train['Customer_Segment'].to_numpy() # Agarra la columna 'Customer_Segment' y la convierte en un np array
Y_test = Y_test['Customer_Segment'].to_numpy()

# Calculo el Y estimado
Y_estimado = KNN(T_test, T_train, Y_train, K)

# Calculo el error
error = 0
for i in range(len(Y_test)):
    if Y_test[i] != Y_estimado[i]:
        error += 1
error = error/len(Y_test)

print(error)

0.05555555555555555


In [85]:
def clasif_PCA(clasif, X_train, Y_train, CP, K):
    X_train_norm, train_mean, train_s = normalizacion(X_train)
    clasif_norm = (clasif - train_mean) / train_s

    X_cov = (X_train_norm.T @ X_train_norm) / len(X_train_norm)
    Avals, Avecs = todos_Avals_Avecs(X_cov, X_cov.shape[0])

    # Sea V  la matriz con los autovectores por columnas  
    V = np.array(Avecs).T

    # Calculo las componentes principales
    T_train = (X_train_norm @ V[:,:CP])
    T_clasif = (clasif_norm @ V[:,:CP])

    return KNN(T_clasif, T_train, Y_train, K), Avals[:CP]


In [35]:
from fractions import Fraction

In [79]:
#ejercicio 2
X = df_wine_independientes.values
Y = df_wine_dependientes['Customer_Segment'].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# clasificación haciendo PCA
Y_estimado, Avals = clasif_PCA(X_test, X_train, Y_train, 4, 5)

desaciertos = 0
for i in range(len(Y_test)):
    if Y_test[i] != Y_estimado[i]:
        desaciertos += 1
        total =len(Y_test)
error = desaciertos/total


df = pd.DataFrame({'estimados':Y_estimado, 'reales': Y_test.T})

print(f'{desaciertos} / {total} = {error}')
df

2 / 36 = 0.05555555555555555


,estimados,reales
0,2,2
1,1,1
2,2,2
3,2,2
4,1,2
5,1,1
6,3,2
7,1,1
8,3,3
9,2,2


en el caso 17 no le pegó

In [114]:
#(a)
df = pd.DataFrame(columns=['Modelo PCA','Componente', 'Varianza explicada', 'Porcentaje', 'Acumulado'])

# Llenar los valores faltantes
matriz_Avals = []
for CP in range(1,5):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    # clasificación haciendo PCA
    Y_estimado, Avals = clasif_PCA(X_test, X_train, Y_train, CP, 5)
    matriz_Avals.append(Avals)

df.loc[0] = [1, 1, matriz_Avals[0][0]/sum(matriz_Avals[0]), (matriz_Avals[0][0]/sum(matriz_Avals[0]))*100, (matriz_Avals[0][0]/sum(matriz_Avals[0]))*100]

df.loc[1] = [2, 1,matriz_Avals[1][0]/sum(matriz_Avals[1]), (matriz_Avals[1][0]/sum(matriz_Avals[1]))*100, (matriz_Avals[1][0]/sum(matriz_Avals[1]))*100]
df.loc[2] = [2, 2,matriz_Avals[1][1]/sum(matriz_Avals[1]), (matriz_Avals[1][1]/sum(matriz_Avals[1]))*100, (matriz_Avals[1][0]/sum(matriz_Avals[1]))*100 + (matriz_Avals[1][1]/sum(matriz_Avals[1]))*100]

df.loc[3] = [3, 1,matriz_Avals[2][0]/sum(matriz_Avals[2]), (matriz_Avals[2][0]/sum(matriz_Avals[2]))*100, (matriz_Avals[2][0]/sum(matriz_Avals[2]))*100]
df.loc[4] = [3, 2, matriz_Avals[2][1]/sum(matriz_Avals[2]), (matriz_Avals[2][1]/sum(matriz_Avals[2]))*100, (matriz_Avals[2][1]/sum(matriz_Avals[2]))*100 + (matriz_Avals[2][0]/sum(matriz_Avals[2]))*100]
df.loc[5] = [3, 3, matriz_Avals[2][2]/sum(matriz_Avals[2]), (matriz_Avals[2][2]/sum(matriz_Avals[2]))*100, (matriz_Avals[2][2]/sum(matriz_Avals[2]))*100 + (matriz_Avals[2][0]/sum(matriz_Avals[2]))*100 + (matriz_Avals[2][1]/sum(matriz_Avals[2]))*100]

df.loc[6] = [4, 1, matriz_Avals[3][0]/sum(matriz_Avals[3]), (matriz_Avals[3][0]/sum(matriz_Avals[3]))*100, (matriz_Avals[3][0]/sum(matriz_Avals[3]))*100]
df.loc[7] = [4, 2, matriz_Avals[3][1]/sum(matriz_Avals[3]), (matriz_Avals[3][1]/sum(matriz_Avals[3]))*100, (matriz_Avals[3][1]/sum(matriz_Avals[3]))*100 + (matriz_Avals[3][0]/sum(matriz_Avals[3]))*100]
df.loc[8] = [4, 3, matriz_Avals[3][2]/sum(matriz_Avals[3]), (matriz_Avals[3][2]/sum(matriz_Avals[3]))*100, (matriz_Avals[3][2]/sum(matriz_Avals[3]))*100 + (matriz_Avals[3][0]/sum(matriz_Avals[3]))*100 +(matriz_Avals[3][1]/sum(matriz_Avals[3]))*100]
df.loc[9] = [4, 4, matriz_Avals[3][3]/sum(matriz_Avals[3]), (matriz_Avals[3][3]/sum(matriz_Avals[3]))*100, (matriz_Avals[3][3]/sum(matriz_Avals[3]))*100 + (matriz_Avals[3][0]/sum(matriz_Avals[3]))*100 +(matriz_Avals[3][1]/sum(matriz_Avals[3]))*100 + (matriz_Avals[3][2]/sum(matriz_Avals[3]))*100]


# Imprimir el dataframe
print(df)

   Modelo PCA  Componente  Varianza explicada  Porcentaje   Acumulado
0         1.0         1.0            1.000000  100.000000  100.000000
1         2.0         1.0            0.659674   65.967413   65.967413
2         2.0         2.0            0.340326   34.032587  100.000000
3         3.0         1.0            0.541636   54.163572   54.163572
4         3.0         2.0            0.276812   27.681242   81.844813
5         3.0         3.0            0.181552   18.155187  100.000000
6         4.0         1.0            0.486312   48.631247   48.631247
7         4.0         2.0            0.261040   26.103989   74.735236
8         4.0         3.0            0.148169   14.816878   89.552114
9         4.0         4.0            0.104479   10.447886  100.000000
